# Сreating tsv data to generate cloned voices based

In [7]:
import pandas as pd
from enum import Enum
import numpy as np

In [8]:
class DataFrameColumn(str, Enum):
    path = "path"
    client_id = "client_id"
    sentence = "sentence"
    generated_sentence = "generated_sentence"
    generated_path = "generated_path"

In [9]:
class GeneratedObjcet:
    def __init__(self, path, client_id, sentence,  generated_path=None, generated_sentence=None):
        self.path = path
        self.client_id = client_id
        self.generated_path = generated_path
        self.sentence = sentence
        self.generated_sentence = generated_sentence

In [10]:
original_string = "common_voice_en_37476269"
new_string = original_string.replace("common_voice_en", "generated_voice_en")

In [11]:
def select_rundom_client_sample(df, client_id):
  df_filtered = df[df[DataFrameColumn.client_id] != client_id]
  return df_filtered.sample(n=1)

def generate_new_path(original_path):
  return original_path.replace("common_voice_en", "generated_voice_en")


In [12]:
def generate_voice_tsv(original_voice_tsv,  generated_voice_tsv):
  tsv_file_path = original_voice_tsv
  df = pd.read_csv(tsv_file_path, sep='\t')
  voices = []
  for _, row in df.iterrows():
      client_id=row[DataFrameColumn.client_id],
      sample = select_rundom_client_sample(df, client_id)
      path=row[DataFrameColumn.path]
      generated_path=generate_new_path(path)
      voices.append(
        GeneratedObjcet(
          path=path,
          client_id=row[DataFrameColumn.client_id],
          sentence=row[DataFrameColumn.sentence],
          generated_sentence=sample[DataFrameColumn.sentence].values[0],
          generated_path=generated_path,
        )
      )
  generate_df =pd.DataFrame([vars(generated_voice) for generated_voice in voices])
  generate_df.to_csv(generated_voice_tsv, sep="\t", index=False)


In [13]:
np.random.seed(200)
original_voice_tsv = "./data/validated.tsv"
generated_voice_tsv = "./data/generated.tsv"
generate_voice_tsv(original_voice_tsv, generated_voice_tsv)